TODO
X Add a more secure way to handle the API keys - they Azure Keypass manager gave me a permission denied.
X Decide how I want to get data? At the moment, none, let's see how Chat-GPTo works
- What are the features I want? A chatbot that helps me filter bikes and gives me links to the bike website
- Add a user interface : simple webapp with dots when chatting
- Share it on Github
- Share it on my website
- Test it with various testing options
  

In [1]:
!pip install langchain_community
!pip install -qU langchain-openai

In [2]:
# from langchain_community.llms import HuggingFaceHub deprecated
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chains import LLMChain  # need to check if this is deprecated
from langchain_core.prompts import PromptTemplate

# So we can get the API keys we need, but need to figure out a better way
#from getpass import getpass



In [3]:
# set the environment variables
import os

print(os.environ["AZURE_OPENAI_API_KEY"])

362c96e85fd544bab449025f9f6a60a0


In [4]:
from langchain_core.messages import HumanMessage
from langchain.chat_models import AzureChatOpenAI

model = AzureChatOpenAI(
    # openai_api_type="azure",
    deployment_name="gpt-4o", 
    model_name="gpt-4o") 



/Users/rebecca/miniconda3/envs/python3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
''' 
Play around with some prompt engineering and look at the different types of responses.
'''
'''
system_message = """
Recommend mountain bikes that fit the criteria. In a bulleted list, name the bike brand, model, frame size, and standover height.
"""
template = ChatPromptTemplate.from_messages([
    ("system", "You are a {product} recommender system. Only answer questions about {product}. Provide the response as a list of {product}, the standover height and a link to the website about that bike."),
    ("human", "Hello, I'm interested in a {product}"),
    ("ai", "Sure, what are your requirements for a {product}!"),
    ("human", "{user_input}"),
])

prompt_value = template.invoke(
    {
        "product": "Mountain Bike",
        "user_input": "I want a fully and I'm 157cm tall."
    }
)
model.invoke(prompt_value)
'''

'\nsystem_message = """\nRecommend mountain bikes that fit the criteria. In a bulleted list, name the bike brand, model, frame size, and standover height.\n"""\ntemplate = ChatPromptTemplate.from_messages([\n    ("system", "You are a {product} recommender system. Only answer questions about {product}. Provide the response as a list of {product}, the standover height and a link to the website about that bike."),\n    ("human", "Hello, I\'m interested in a {product}"),\n    ("ai", "Sure, what are your requirements for a {product}!"),\n    ("human", "{user_input}"),\n])\n\nprompt_value = template.invoke(\n    {\n        "product": "Mountain Bike",\n        "user_input": "I want a fully and I\'m 157cm tall."\n    }\n)\nmodel.invoke(prompt_value)\n'

Prompt "I am short"
So that example is ok, it largely shows women bikes.  It misses, for example, Pivot and Ibis, both which are high-end mountain bikes known for having smaller sizes.
I'll feel more confident if it shows my bike and Roxy's bikes in the results
https://roxybikemallorca.com/gebraucht-radverkauf-mondraker/

Prompt: I want a fully and I'm 157cm tall. 
" **Specialized Stumpjumper Comp Alloy 29**\"

# this is where I finished on Tuesday
# I think the next steps are to build a chatbot
# https://python.langchain.com/v0.2/docs/tutorials/chatbot/
# https://python.langchain.com/v0.2/docs/tutorials/rag/

# I need to improve the API key handling and I need to add this to github https://github.com/rbalebako/genAI-notebooks

In [6]:
'''
We are building a chatbot to help narrow down the types of bikes someone might want.
We can use a Message History class to wrap our model and make it stateful. 
This will keep track of inputs and outputs of the model, and store them in some datastore.
'''

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [7]:
# we will want to pass a new session id to the chatbot for every new user or session
config = {"configurable": {"session_id": "abc2"}}

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a mountain bike recommender system. Only answer questions about mountain bikes.
            Provide the response as a list with the brand name, the model name, the standover height, and link to the website about that bike.
            """,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [12]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm looking for a fully mountain bike.")]})

response.content

'Sure! Here are some fully mountain bikes you might be interested in:\n\n1. **Brand:** Trek\n   **Model:** Fuel EX 9.8\n   **Standover Height:** 76.2 cm (30 inches)\n   **Link:** [Trek Fuel EX 9.8](https://www.trekbikes.com/us/en_US/bikes/mountain-bikes/trail-mountain-bikes/fuel-ex/fuel-ex-9-8/p/33061/)\n\n2. **Brand:** Specialized\n   **Model:** Stumpjumper Expert\n   **Standover Height:** 78.5 cm (30.9 inches)\n   **Link:** [Specialized Stumpjumper Expert](https://www.specialized.com/us/en/stumpjumper-expert/p/175269)\n\n3. **Brand:** Giant\n   **Model:** Trance Advanced Pro 29 1\n   **Standover Height:** 75.6 cm (29.8 inches)\n   **Link:** [Giant Trance Advanced Pro 29 1](https://www.giant-bicycles.com/us/trance-advanced-pro-29-1)\n\n4. **Brand:** Santa Cruz\n   **Model:** Hightower\n   **Standover Height:** 74.4 cm (29.3 inches)\n   **Link:** [Santa Cruz Hightower](https://www.santacruzbicycles.com/en-US/bikes/hightower)\n\n5. **Brand:** Canyon\n   **Model:** Spectral 29 CF 8\n   *

In [ ]:
# Limit the size of the message to avoid overflow
# BEFORE the prompt template but AFTER you load previous messages from Message History.

from langchain_core.runnables import RunnablePassthrough


def filter_messages(messages, k=10):
    return messages[-k:]


chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

In [ ]:
#Now that we've built an application, we need to serve it. That's where LangServe comes in. LangServe helps developers deploy LangChain chains as a REST API. You do not need to use LangServe to use LangChain, but in this guide we'll show how you can deploy your app with LangServe.
#https://python.langchain.com/v0.2/docs/tutorials/llm_chain/